# Загрузка Pandas и очистка данных

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('main_task_new.csv')
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [4]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [5]:
#приводим в опрятный вид название колонок, убираем лищние символы из id и id_ta,
#приводим к нумерическому типу
df.columns = ['id', 'city', 'cuisine_style', 'ranking', 'rating', 'price_range', 'number_of_reviews', 'reviews', 'url_ta', 'id_ta']
df.id_ta = df.id_ta.str.replace('d', '')
df.id = df.id.str.replace('id_','')
df.id = pd.to_numeric(df.id)
df.id_ta = pd.to_numeric(df.id_ta)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 40000 non-null  int64  
 1   city               40000 non-null  object 
 2   cuisine_style      30717 non-null  object 
 3   ranking            40000 non-null  float64
 4   rating             40000 non-null  float64
 5   price_range        26114 non-null  object 
 6   number_of_reviews  37457 non-null  float64
 7   reviews            40000 non-null  object 
 8   url_ta             40000 non-null  object 
 9   id_ta              40000 non-null  int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 3.1+ MB


In [6]:
# посмотрим на ценовой диапазон:
df.price_range.value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: price_range, dtype: int64

In [7]:
#переводим price_range в цифровые значения, задав соответствующий словарь

df['price_range'] = df['price_range'].fillna(0)
price_range_dict = {}
price_range_dict = { '0':0, '$':1, '$$ - $$$':2, '$$$$':3  }
df['price_range']= df['price_range'].replace(to_replace=price_range_dict)
df['price_range'].value_counts()

2    18412
0    13886
1     6279
3     1423
Name: price_range, dtype: int64

In [8]:
#приведем в порядок столбец с кухнями:
df['cuisine_style']= df['cuisine_style'].str.replace('[', '')
df['cuisine_style']= df['cuisine_style'].str.replace(']', '')
df['cuisine_style']= df['cuisine_style'].str.replace("'", '')
df['cuisine_style']= df['cuisine_style'].fillna("Unknown")
df['cuisine_style'] = df['cuisine_style'].str.split(', ')

df['cuisine_style']

0                        [European, French, International]
1                                                [Unknown]
2        [Japanese, Sushi, Asian, Grill, Vegetarian Fri...
3                                                [Unknown]
4          [German, Central European, Vegetarian Friendly]
                               ...                        
39995    [Italian, Vegetarian Friendly, Vegan Options, ...
39996    [French, American, Bar, European, Vegetarian F...
39997                                    [Japanese, Sushi]
39998    [Polish, European, Eastern European, Central E...
39999                                            [Spanish]
Name: cuisine_style, Length: 40000, dtype: object

In [9]:
#добавим столбец с количеством кухонь
df['cuis_num'] = df.cuisine_style.apply(len)
df.cuis_num

0        3
1        1
2        7
3        1
4        3
        ..
39995    4
39996    5
39997    2
39998    5
39999    1
Name: cuis_num, Length: 40000, dtype: int64

In [10]:
#переходим к работе с reviews. Выделим в отдельный столбец date_rev даты всех ревью
import re
pattern = re.compile("\d+\/\d+\/\d+")
df["date_rev"] = df["reviews"].apply(pattern.findall)
display(df["date_rev"].head())

0    [12/31/2017, 11/20/2017]
1    [07/06/2017, 06/19/2016]
2    [01/08/2018, 01/06/2018]
3                          []
4    [11/18/2017, 02/19/2017]
Name: date_rev, dtype: object

In [11]:
#отсортируем даты, чтобы первый элемент был во времени раньше, чем второй:
df["date_rev"] = df["date_rev"].apply(pd.to_datetime).apply(sorted)
df.date_rev


0        [2017-11-20 00:00:00, 2017-12-31 00:00:00]
1        [2016-06-19 00:00:00, 2017-07-06 00:00:00]
2        [2018-01-06 00:00:00, 2018-01-08 00:00:00]
3                                                []
4        [2017-02-19 00:00:00, 2017-11-18 00:00:00]
                            ...                    
39995    [2017-11-12 00:00:00, 2017-12-16 00:00:00]
39996    [2017-12-12 00:00:00, 2017-12-21 00:00:00]
39997    [2008-04-12 00:00:00, 2016-11-03 00:00:00]
39998    [2017-06-18 00:00:00, 2017-07-11 00:00:00]
39999    [2012-07-04 00:00:00, 2016-01-31 00:00:00]
Name: date_rev, Length: 40000, dtype: object

In [12]:
#Разделим столбец date_rev на два с двумя датами ревью:
df["rev_1"] = df["date_rev"].apply(lambda x: x[0] if len(x)>=1 else None)
df["rev_2"] = df["date_rev"].apply(lambda x: x[1] if len(x)==2 else None)
df.head()

,id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta,cuis_num,date_rev,rev_1,rev_2
0,5569,Paris,"[European, French, International]",5570.0,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,1912643,3,"[2017-11-20 00:00:00, 2017-12-31 00:00:00]",2017-11-20,2017-12-31
1,1535,Stockholm,[Unknown],1537.0,4.0,0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,7992032,1,"[2016-06-19 00:00:00, 2017-07-06 00:00:00]",2016-06-19,2017-07-06
2,352,London,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",353.0,4.5,3,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,8632781,7,"[2018-01-06 00:00:00, 2018-01-08 00:00:00]",2018-01-06,2018-01-08
3,3456,Berlin,[Unknown],3458.0,5.0,0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,1358776,1,[],NaT,NaT
4,615,Munich,"[German, Central European, Vegetarian Friendly]",621.0,4.0,2,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,6864963,3,"[2017-02-19 00:00:00, 2017-11-18 00:00:00]",2017-02-19,2017-11-18


In [13]:
#добавим столбец с количеством дней с последнего ревью:
df['days'] = pd.to_datetime(df.rev_2).sub(pd.Timestamp('2021-07-14')).dt.days
df['days'] = abs(df.days)
df['days']= df['days'].fillna(0)
df['days']

0        1291.0
1        1469.0
2        1283.0
3           0.0
4        1334.0
          ...  
39995    1306.0
39996    1301.0
39997    1714.0
39998    1464.0
39999    1991.0
Name: days, Length: 40000, dtype: float64

In [14]:
#добавим столбец с количеством дней (разницей) между первым и вторым ревью:
df['days_dif'] = pd.to_datetime(df.rev_2).sub(df.rev_1).dt.days
df['days_dif'] = df['days_dif'].fillna(0)

In [15]:
#в столбце review - ноль нулевых значений, при этом в столбце количество 
#ревью они есть. Очевидно, это пропуски. Заполним их не нулевым значением (1):

def number_reviews(row):
    if pd.isnull(row['number_of_reviews']):
        if row['reviews'] is None:
            return 0
        else:
            return 1
    return row['number_of_reviews']

df['number_of_reviews'] = df.apply(number_reviews, axis = 1)



In [17]:
#преобразуем столбец city в dummy-переменные:
df.city = df.city.apply(lambda x: x.lower())
df4=df.copy() #делаю копию df на случай, если понадобится откатить изменения

df4 = pd.get_dummies(df4, columns=['city'])
df4.head()

,id,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta,cuis_num,...,city_munich,city_oporto,city_oslo,city_paris,city_prague,city_rome,city_stockholm,city_vienna,city_warsaw,city_zurich
0,5569,"[European, French, International]",5570.0,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,1912643,3,...,0,0,0,1,0,0,0,0,0,0
1,1535,[Unknown],1537.0,4.0,0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,7992032,1,...,0,0,0,0,0,0,1,0,0,0
2,352,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",353.0,4.5,3,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,8632781,7,...,0,0,0,0,0,0,0,0,0,0
3,3456,[Unknown],3458.0,5.0,0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,1358776,1,...,0,0,0,0,0,0,0,0,0,0
4,615,"[German, Central European, Vegetarian Friendly]",621.0,4.0,2,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,6864963,3,...,1,0,0,0,0,0,0,0,0,0


In [18]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 46 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 40000 non-null  int64         
 1   cuisine_style      40000 non-null  object        
 2   ranking            40000 non-null  float64       
 3   rating             40000 non-null  float64       
 4   price_range        40000 non-null  int64         
 5   number_of_reviews  40000 non-null  float64       
 6   reviews            40000 non-null  object        
 7   url_ta             40000 non-null  object        
 8   id_ta              40000 non-null  int64         
 9   cuis_num           40000 non-null  int64         
 10  date_rev           40000 non-null  object        
 11  rev_1              33529 non-null  datetime64[ns]
 12  rev_2              28971 non-null  datetime64[ns]
 13  days               40000 non-null  float64       
 14  days_d

In [19]:
#удалим не-числовые столбцы

df4 = df4.drop(['cuisine_style', 'reviews', 'url_ta', 'date_rev', 'rev_1', 
          'rev_2'], axis=1)
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 40 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 40000 non-null  int64  
 1   ranking            40000 non-null  float64
 2   rating             40000 non-null  float64
 3   price_range        40000 non-null  int64  
 4   number_of_reviews  40000 non-null  float64
 5   id_ta              40000 non-null  int64  
 6   cuis_num           40000 non-null  int64  
 7   days               40000 non-null  float64
 8   days_dif           40000 non-null  float64
 9   city_amsterdam     40000 non-null  uint8  
 10  city_athens        40000 non-null  uint8  
 11  city_barcelona     40000 non-null  uint8  
 12  city_berlin        40000 non-null  uint8  
 13  city_bratislava    40000 non-null  uint8  
 14  city_brussels      40000 non-null  uint8  
 15  city_budapest      40000 non-null  uint8  
 16  city_copenhagen    400

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [20]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df4.drop(['id', 'rating'], axis = 1)
y = df4['rating']

In [21]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [22]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [23]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [24]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [25]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21549300000000002


In [ ]:
#Искомая точность модели, "как у автора" получена